In [1]:
import os
import numpy as np
import pandas as pd
import time
import scanpy as sc
import torch

import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO)

import sys
sys.path.append('../../')
import scDualGN
sys.path.append('../')
import expri_helper

/jdfssz1/ST_HEALTH/P18Z10200N0124/AI/user/baiyong/anaconda3/envs/mytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dat_path = expri_helper.get_path('ren_scdualgn', dat_ls_file='../expri_data.json')
adata = sc.read_h5ad(dat_path)

##################

# del adata.obsm['X_tsne'],adata.obsm['X_pca'],adata.obsm['har_emb']
# adata.obs['celltype'] = adata.obs['majorType']
# del adata.obs['majorType']
# adata.obs['celltype_str'] = adata.obs['celltype']
# cell_type_dict = {'B':0, 'CD8':1, 'Mono':2, 'CD4':3, 'NK':4, 'Macro':5, 'DC':6, 'Plasma':7, 'Mega':8, 'Epi':9, 'Neu':10, 'Mast':11} 
# adata.obs['celltype'] = adata.obs['celltype'].map(cell_type_dict)

In [5]:
adata

AnnData object with n_obs × n_vars = 1462702 × 2500
    obs: 'celltype', 'majorType', 'sampleID', 'PatientID', 'datasets', 'City', 'Age', 'Sex', 'Sample type', 'CoVID-19 severity', 'Sample time', 'Sampling day (Days after symptom onset)', 'SARS-CoV-2', 'Single cell sequencing platform', 'BCR single cell sequencing', 'TCR single cell sequencing', 'Outcome', 'Comorbidities', 'COVID-19-related medication and anti-microbials', 'Leukocytes [G/L]', 'Neutrophils [G/L]', 'Lymphocytes [G/L]', 'Unpublished', 'celltype_str', 'n_counts'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'neighbors', 'pca'
    obsp: 'connectivities', 'distances'

In [3]:
#######################
# sc.pp.filter_cells(adata, min_counts=100)
# sc.pp.filter_genes(adata, min_cells=3)
# sc.pp.normalize_total(adata,target_sum=1e4)
# sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, n_top_genes=2500)
# adata = adata[:, adata.var.highly_variable]
# sc.pp.scale(adata,max_value=10)

In [6]:
#kmeans
# start = time.time()
# z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata, n_cluster=18,n_z=32,n_epoch_update_pq=5, n_epochs=30,alpha_dualvae=0.02,beta_daulvae=5,gamma_dualvae=4,GPU_id=2)

# scdualgn_model = scDualGN.scDualGN(adata, n_z=32,device='cuda:2', verbosity=True, batch_size=1024).pretrain(alpha=0.05, beta=20, gamma=4)
# scdualgn_model.cluster(n_cluster=18, eta=1, nu=1, n_epochs=30)

lr, alpha, gamma, nu = 0.0001, 0.01, 4, 0.01
print('=====================================')
print('lr:{}, alpha:{}, gamma:{}, nu:{}'.format(lr, alpha, gamma, nu))
start = time.time()
torch.cuda.empty_cache()
scdualgn_model = scDualGN.scDualGN(adata, n_z=32,device='cuda:1', batch_size=1024*2, verbosity=True, lr=lr).pretrain(alpha=alpha, beta=1, gamma=gamma)
scdualgn_model.cluster(n_cluster=12, eta=1, nu=nu, n_epochs=32)


end = time.time()

print('running time = {}'.format(end-start))

INFO:scDualGN.run:dual-VAE pretrain start...


lr:0.0001, alpha:0.01, gamma:4, nu:0.01


INFO:scDualGN.run:Epoch 1/50,Overall loss:1861.4943,MSE:1317.3462,MSE1:135.7801,KL: 102.7807
INFO:scDualGN.run:Epoch 2/50,Overall loss:1074.5534,MSE:1045.0495,MSE1:7.0007,KL: 150.1067
INFO:scDualGN.run:Epoch 3/50,Overall loss:1042.7901,MSE:1027.8572,MSE1:3.3052,KL: 171.2048
INFO:scDualGN.run:Epoch 4/50,Overall loss:1029.7287,MSE:1019.4764,MSE1:2.1009,KL: 184.8735
INFO:scDualGN.run:Epoch 5/50,Overall loss:1021.1629,MSE:1012.8618,MSE1:1.5923,KL: 193.1805
INFO:scDualGN.run:Epoch 6/50,Overall loss:1013.8199,MSE:1006.4563,MSE1:1.3476,KL: 197.3131
INFO:scDualGN.run:Epoch 7/50,Overall loss:1006.7716,MSE:999.8346,MSE1:1.2395,KL: 197.8906
INFO:scDualGN.run:Epoch 8/50,Overall loss:1000.4552,MSE:993.7241,MSE1:1.1900,KL: 197.1189
INFO:scDualGN.run:Epoch 9/50,Overall loss:996.3282,MSE:989.7653,MSE1:1.1491,KL: 196.6556
INFO:scDualGN.run:Epoch 10/50,Overall loss:994.4438,MSE:987.9988,MSE1:1.1196,KL: 196.6484
INFO:scDualGN.run:Epoch 11/50,Overall loss:980.2129,MSE:975.0457,MSE1:0.8028,KL: 195.6008
INF

acc 0.6957, nmi 0.6907, ari 0.6302, hs 0.7301, cs 0.6553, purity: 0.7932
running time = 4490.2222464084625


In [ ]:
#leiden
# start = time.time()
# z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata,n_z=32,n_epoch_update_pq=5, n_epochs=30,alpha_dualvae=0.02,beta_daulvae=5,gamma_dualvae=4,GPU_id=2,n_neighbors=50)
# end = time.time()

# print('running time = {}'.format(end-start))